In [80]:
import pandas as pd
import openai
import time

In [60]:
path = ''

In [61]:
wine_list = pd.read_excel(path)

In [62]:
wine_list

,REGIÃO,TIPO,PRODUTOR,CÓDIGO,PRODUTO,COLHEITA,CAPACIDADE (L),GFS/CX,AVALIAÇÃO,PREÇO PRIVADO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DOMAINE FAIVELEY - FRANÇA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BORGONHA,BRANCO,FAIVELEY,23417.0,Bourgogne Chardonnay,2020,0.75,6.0,90 JS,33.0
3,BORGONHA,BRANCO,FAIVELEY,23421.0,Montagny Blanc,2018,0.75,6.0,NaN,29.0
4,BORGONHA,TINTO,FAIVELEY,23234.0,Bourgogne Pinot Noir,2020,0.75,6.0,91 JS,34.0
...,...,...,...,...,...,...,...,...,...,...
72,VALLE DE UCO,BRANCO,ZUCCARDI,23402.0,Semillon Poligonos Tupungato,2021,0.75,6.0,94 JS,33.0
73,VALLE DE UCO,TINTO,ZUCCARDI,23395.0,Malbec Série A,2021,0.75,6.0,92 JS,16.0
74,VALLE DE UCO,TINTO,ZUCCARDI,23397.0,"Malbec ""Q""",2020,0.75,6.0,94 JS,25.0
75,VALLE DE UCO,TINTO,ZUCCARDI,23398.0,"Cabernet Franc ""Q""",2020,0.75,6.0,94 JS,25.0


In [63]:
cols = ['REGIÃO', 'TIPO', 'PRODUTOR', 'PRODUTO', 'COLHEITA']

In [64]:
wine_list['FULL_TITLE'] = wine_list[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [65]:
wine_list.dropna(subset=['PRODUTO'], inplace=True)

In [66]:
wine_list.notna().sum()

REGIÃO            55
TIPO              55
PRODUTOR          55
CÓDIGO            55
PRODUTO           55
COLHEITA          55
CAPACIDADE (L)    55
GFS/CX            53
AVALIAÇÃO         24
PREÇO PRIVADO     55
FULL_TITLE        55
dtype: int64

In [67]:
wine_list.head()

,REGIÃO,TIPO,PRODUTOR,CÓDIGO,PRODUTO,COLHEITA,CAPACIDADE (L),GFS/CX,AVALIAÇÃO,PREÇO PRIVADO,FULL_TITLE
2,BORGONHA,BRANCO,FAIVELEY,23417.0,Bourgogne Chardonnay,2020,0.75,6.0,90 JS,33.0,BORGONHA BRANCO FAIVELEY Bourgogne Chardonnay ...
3,BORGONHA,BRANCO,FAIVELEY,23421.0,Montagny Blanc,2018,0.75,6.0,NaN,29.0,BORGONHA BRANCO FAIVELEY Montagny Blanc 2018
4,BORGONHA,TINTO,FAIVELEY,23234.0,Bourgogne Pinot Noir,2020,0.75,6.0,91 JS,34.0,BORGONHA TINTO FAIVELEY Bourgogne Pinot Noir 2020
6,CHABLIS,BRANCO,LOUIS MICHEL,23571.0,Petit Chablis,2022,0.75,6.0,NaN,29.0,CHABLIS BRANCO LOUIS MICHEL Petit Chablis 2022
7,CHABLIS,BRANCO,LOUIS MICHEL,23572.0,Chablis,2021,0.75,6.0,NaN,37.0,CHABLIS BRANCO LOUIS MICHEL Chablis 2021


In [68]:
wine_list['DESCRIPTION'] = None

In [69]:
wine_list['DESCRIPTION_rus'] = None

In [73]:
wine_list.tail()

,REGIÃO,TIPO,PRODUTOR,CÓDIGO,PRODUTO,COLHEITA,CAPACIDADE (L),GFS/CX,AVALIAÇÃO,PREÇO PRIVADO,FULL_TITLE,DESCRIPTION,DESCRIPTION_rus
72,VALLE DE UCO,BRANCO,ZUCCARDI,23402.0,Semillon Poligonos Tupungato,2021,0.75,6.0,94 JS,33.0,VALLE DE UCO BRANCO ZUCCARDI Semillon Poligono...,None,None
73,VALLE DE UCO,TINTO,ZUCCARDI,23395.0,Malbec Série A,2021,0.75,6.0,92 JS,16.0,VALLE DE UCO TINTO ZUCCARDI Malbec Série A 2021,None,None
74,VALLE DE UCO,TINTO,ZUCCARDI,23397.0,"Malbec ""Q""",2020,0.75,6.0,94 JS,25.0,"VALLE DE UCO TINTO ZUCCARDI Malbec ""Q"" 2020",None,None
75,VALLE DE UCO,TINTO,ZUCCARDI,23398.0,"Cabernet Franc ""Q""",2020,0.75,6.0,94 JS,25.0,"VALLE DE UCO TINTO ZUCCARDI Cabernet Franc ""Q""...",None,None
76,VALLE DE UCO,TINTO,ZUCCARDI,23399.0,"Cabernet Sauvignon ""Q""",2020,0.75,6.0,94 JS,25.0,VALLE DE UCO TINTO ZUCCARDI Cabernet Sauvignon...,None,None


In [93]:
def get_description(wine_name, lang):
    api_key = '....'
    
    wine_template = "type (like red dry or white sweet or smth else), alcohol, grapes, style, flavour, food goes with, temperature serve"
    cht_req = f"""give me description of this wine {wine_name} in {lang} according to this template: {wine_template}"""
    
    openai.api_key = api_key

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Вы - пользователь"},
            {"role": "user", "content": cht_req}
        ],
        max_tokens=350,  # Ограничение количества токенов в ответе
        n=1,  # Получить 1 альтернативных ответа
        stop=None,  # Модель остановится после этой фразы
        temperature=0.3  # Установка температуры для контроля случайности
                         #1 - более случайной и креативной; 0 - одель будет более предсказуемой и фокусированной
        )

    
    return response.choices[0].message.content


In [81]:
for i in wine_list.index:
    wine_name = wine_list.loc[i, 'FULL_TITLE']
    wine_list.loc[i, 'DESCRIPTION'] = get_description(wine_name, 'english')
    wine_list.loc[i, 'DESCRIPTION_rus'] = get_description(wine_name, 'russian')
    time.sleep(60)

In [84]:
wine_list.to_excel(#outfile)  